# MWS Secondary Target Selection

## MWS_SPECIAL_DRACO

Maintainers:
  - Mei-Yu Wang (selection/catalog)
  - Andrew Cooper (secondary target code)

The catalog (`Draco_TS.fits`) contains the Gaia source id, ra, dec (in degrees), and two columns showing whether they are observed by APOGEE or Walker et al. (2015) (arXiv:1503.02589). 

The selection criteria are:

   1. Proper motion cut of |pm0-pm|<=2.0. The Draco PM is from Fritz et al. 2018 
   
   2. CMD cut (isochrone+distance), with delta(g-r) < 0.15 to a isochrone (with age= 12 Gyr and [Fe/H] =-1.7 ) with a magnitude cut at Gaia G < 20. 
      
Note that these cuts are based only on Gaia photometry.

In [1]:
pwd

'/global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_DRACO_SV/1/code'

In [7]:
ls -l ../raw

total 128
-rw-r--r-- 1 apcooper apcooper 63360 Aug 11 21:48 Draco_TS.fits


In [8]:
import os
import numpy as np
from astropy.table import Table, Column
import astropy.io.fits as fits

In [9]:
target_class   = 'MWS_SPECIAL_DRACO_SV'
input_filename = 'Draco_TS.fits'

REF_CAT   = 'G'
REF_EPOCH = '2015.5'

In [10]:
# Relative paths
input_path  = os.path.join('..','raw',input_filename)
output_path = os.path.join('..','indata','{}.fits'.format(target_class))

output_dir  = os.path.dirname(output_path)
if not os.path.exists(output_dir):
    print('Making output directory: {}'.format(os.path.abspath(output_dir)))
    os.makedirs(dir)

In [11]:
print('Reading input: {}'.format(os.path.abspath(input_path)))
d = Table.read(input_path)
print('Read {:d} rows'.format(len(d)))

Reading input: /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_DRACO_SV/1/raw/Draco_TS.fits
Read 2195 rows


In [12]:
print('First 3 rows:')
print(d[0:3])

First 3 rows:
      gaia_id             ra           dec      APOGEE Walker+2015
------------------- ------------- ------------- ------ -----------
1421335245252389888 264.753746576 56.3708990437  False       False
1421336172965349888 264.762883559 56.4132745436  False       False
1421339746378180736 264.747744927 56.4814987678  False       False


Adjust to secondary target data model:

In [16]:
output = d['ra','dec','gaia_id']

for cn_old, cn_new in zip(['ra','dec','gaia_id'],['RA','DEC','REF_ID']):
    output.rename_column(cn_old,cn_new)

output.add_column(Column(np.repeat(False,len(output)),'OVERRIDE'))
output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMRA'))
output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMDEC'))
output.add_column(Column(np.repeat(REF_CAT,len(output)),'REF_CAT'))
output.add_column(Column(np.repeat(REF_EPOCH,len(output)),'REF_EPOCH'))

Add header:

In [17]:
output.meta['VRAWDATA'] = int(os.path.abspath(input_path).split(os.path.sep)[-3])

Write output:

In [18]:
print('Writing {:d} rows of output to {}'.format(len(output),os.path.abspath(output_path)))
output.write(output_path,overwrite=True)

Writing 2195 rows of output to /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_DRACO_SV/1/indata/MWS_SPECIAL_DRACO_SV.fits
